### Send Cancel Location

#### Connect to SHMDBMMI and collect the subscribers

On services1:

linus> dbs dmi_sdp
   1. slee/dmi_sdp

Which DB to connect ?
1. SHMDB(P)
2. ORACLE
Enter the Nb correponding to the DB type : 1


SHMDBMMI> sub_state.Find(\\$msc>>"4076" && $sponsor_imsi>>"26006")

In [8]:
# python3
# Author: pavel@roamability.com

import re
from bs4 import BeautifulSoup
from os.path import join
from collections import namedtuple
import time
from datetime import datetime
import logging
from importlib import reload
import roamability as rb
reload(logging)

version = '0.1.0'

logger = logging.getLogger('CL')
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

sponsor_object = namedtuple('Sponsor', 'sponsor_imsi_prefix sponsor_gt')


#######################################################################################
# Define variables here
#######################################################################################

downloads = 'C:/Users/balob/Downloads'
file_name = 'dmi_data.txt'

# Chose the Sponsor to whom to send CL

sponsor = sponsor_object('26006', '48790993070')
# sponsor = sponsor_object('42501', '97254120624')

#######################################################################################

pattern  =  re.compile(f',imsi=(\d+).+,msc=(\d*),sgsn=(\d*).+sponsor_imsi=({sponsor.sponsor_imsi_prefix}\d+)')

In [9]:
# Print subscribers

with open(join(downloads, file_name)) as file:
    line = file.read()
    matches = pattern.findall(line)
    
    for i, match in enumerate(matches):
            logger.info(f'{i:04} - RealImsi:{match[0]}; Imsi:{match[3]}; MSC:{match[1]}; SGSN:{match[2]}')

logger.info(f'Total subscribers: {len(matches)}')

2020-01-10 10:19:20,780 - CL - INFO - 0000 - RealImsi:425019613021941; Imsi:260060149021941; MSC:4540699761; SGSN:4528130098
2020-01-10 10:19:20,781 - CL - INFO - 0001 - RealImsi:425019613022446; Imsi:260060149022446; MSC:4540699762; SGSN:4540699702
2020-01-10 10:19:20,782 - CL - INFO - 0002 - RealImsi:425019613006700; Imsi:260060140010417; MSC:4540699761; SGSN:
2020-01-10 10:19:20,783 - CL - INFO - 0003 - RealImsi:240400000030601; Imsi:260060140043350; MSC:4540699762; SGSN:4540699702
2020-01-10 10:19:20,783 - CL - INFO - 0004 - RealImsi:425019613030055; Imsi:260060140122854; MSC:4540699762; SGSN:4540699701
2020-01-10 10:19:20,784 - CL - INFO - 0005 - RealImsi:425019613030364; Imsi:260060140120168; MSC:4540699761; SGSN:4540699701
2020-01-10 10:19:20,785 - CL - INFO - 0006 - RealImsi:724390995549369; Imsi:260060140112280; MSC:4540699762; SGSN:4528130098
2020-01-10 10:19:20,786 - CL - INFO - 0007 - RealImsi:425019613047838; Imsi:260060149046338; MSC:4540699762; SGSN:4540699701
2020-01-10

In [13]:
# Send Cancel Location

start_time = datetime.now()

with open(join(downloads, file_name)) as file:
    line = file.read()
    matches = pattern.findall(line)
    for i, match in enumerate(matches):
        if match[0][1]:
            cl = BeautifulSoup(rb.cl(sponsor.sponsor_gt, match[1], match[3], d_ssn=7),'xml') # ogt, dgt, imsi
            logger.info(f'{i:04} - RealImsi:{match[0]}; Imsi:{match[3]}; MSC:{match[1]}; Result:{cl.result.text}')
        if match[0][2]:
            cl = BeautifulSoup(rb.cl(sponsor.sponsor_gt, match[2], match[3], d_ssn=149),'xml')
            logger.info(f'{i:04} - RealImsi:{match[0]}; Imsi:{match[3]}; SGSN:{match[2]}; Result:{cl.result.text}')
            
logger.info(f'All done. {len(matches)} subscribers processed.\nTotal elapsed time {datetime.now() - start_time}')

2020-01-10 10:29:49,804 - CL - INFO - 0000 - RealImsi:425019613021941; Imsi:260060149021941; MSC:4540699761; Result:0
2020-01-10 10:29:50,485 - CL - INFO - 0000 - RealImsi:425019613021941; Imsi:260060149021941; SGSN:4528130098; Result:0
2020-01-10 10:29:51,096 - CL - INFO - 0001 - RealImsi:425019613022446; Imsi:260060149022446; MSC:4540699762; Result:0
2020-01-10 10:29:51,824 - CL - INFO - 0001 - RealImsi:425019613022446; Imsi:260060149022446; SGSN:4540699702; Result:0
2020-01-10 10:29:52,485 - CL - INFO - 0002 - RealImsi:425019613006700; Imsi:260060140010417; MSC:4540699761; Result:0
2020-01-10 10:30:02,612 - CL - INFO - 0002 - RealImsi:425019613006700; Imsi:260060140010417; SGSN:; Result:-1
2020-01-10 10:30:03,254 - CL - INFO - 0003 - RealImsi:240400000030601; Imsi:260060140043350; MSC:4540699762; Result:0
2020-01-10 10:30:03,997 - CL - INFO - 0003 - RealImsi:240400000030601; Imsi:260060140043350; SGSN:4540699702; Result:0
2020-01-10 10:30:04,651 - CL - INFO - 0004 - RealImsi:4250196